In [305]:
import numpy as np
import pandas as pd
import requests
import json
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import kaleido
import plotly

## Define search engine version

In [306]:
if "SEARCH_VERSION" not in locals():
    SEARCH_VERSION = "new"
else:
    print(SEARCH_VERSION)

2.1


In [307]:
SEARCH_VERSION

'2.1'

## Import lastest elastic data

In [308]:
df_test = pd.read_csv("./data/elastic_wars.csv", dtype=str)

In [309]:
df_test.drop([911], axis=0, inplace=True)

In [310]:
df_test.head(3)

,terms,siren,results_elastic_1.1,pages_elastic_1.1,siren_elastic_1.1,resp_time_elastic_1.1,rank_elastic_1.1,results_elastic_2.0,pages_elastic_2.0,siren_elastic_2.0,resp_time_elastic_2.0,rank_elastic_2.0
0,stellantis,879786085,1383.0,70.0,"['879786085', '803902485', '500019898', '37911...",0.042709,0,2007.0,101.0,"['879786085', '897940953', '912425493', '91005...",0.044964,0
1,STELLANTIS,879786085,1383.0,70.0,"['879786085', '803902485', '500019898', '37911...",0.041482,0,2007.0,101.0,"['879786085', '897940953', '912425493', '91005...",0.046229,0
2,Stellantis,879786085,1383.0,70.0,"['879786085', '803902485', '500019898', '37911...",0.040855,0,2007.0,101.0,"['879786085', '897940953', '912425493', '91005...",0.047226,0


In [311]:
df_test.columns

Index(['terms', 'siren', 'results_elastic_1.1', 'pages_elastic_1.1',
       'siren_elastic_1.1', 'resp_time_elastic_1.1', 'rank_elastic_1.1',
       'results_elastic_2.0', 'pages_elastic_2.0', 'siren_elastic_2.0',
       'resp_time_elastic_2.0', 'rank_elastic_2.0'],
      dtype='object')

In [312]:
df_test.shape

(926, 12)

In [313]:
df_test.dtypes

terms                    object
siren                    object
results_elastic_1.1      object
pages_elastic_1.1        object
siren_elastic_1.1        object
resp_time_elastic_1.1    object
rank_elastic_1.1         object
results_elastic_2.0      object
pages_elastic_2.0        object
siren_elastic_2.0        object
resp_time_elastic_2.0    object
rank_elastic_2.0         object
dtype: object

## Call last search functions (maybe add description)

In [314]:
def find(key, dictionary):
    for k, v in dictionary.items():
        if k == key:
            yield v
        elif isinstance(v, dict):
            for result in find(key, v):
                yield result
        elif isinstance(v, list):
            for d in v:
                for result in find(key, d):
                    yield result

In [315]:
def get_response(url, q):
    params["q"] = q
    response = requests.get(url, params=params)
    time_elapsed = response.elapsed.total_seconds()
    content = json.loads(response.content)
    total_results = content["total_results"]
    total_pages = content["total_pages"]
    siren_list = [unite["siren"] for unite in content["results"]]
    return total_results, total_pages, siren_list, time_elapsed

In [316]:
url_elastic = "http://api.sirene.dataeng.etalab.studio/search"

In [317]:
# Get first 20 results
params = {"q": "", "page": "1", "per_page": "20"}

In [318]:
(
    df_test[f"results_elastic_{SEARCH_VERSION}"],
    df_test[f"pages_elastic_{SEARCH_VERSION}"],
    df_test[f"siren_elastic_{SEARCH_VERSION}"],
    df_test[f"resp_time_elastic_{SEARCH_VERSION}"],
) = ("", "", "", "")

In [319]:
df_test.columns

Index(['terms', 'siren', 'results_elastic_1.1', 'pages_elastic_1.1',
       'siren_elastic_1.1', 'resp_time_elastic_1.1', 'rank_elastic_1.1',
       'results_elastic_2.0', 'pages_elastic_2.0', 'siren_elastic_2.0',
       'resp_time_elastic_2.0', 'rank_elastic_2.0', 'results_elastic_2.1',
       'pages_elastic_2.1', 'siren_elastic_2.1', 'resp_time_elastic_2.1'],
      dtype='object')

In [320]:
df_test.head(3)

,terms,siren,results_elastic_1.1,pages_elastic_1.1,siren_elastic_1.1,resp_time_elastic_1.1,rank_elastic_1.1,results_elastic_2.0,pages_elastic_2.0,siren_elastic_2.0,resp_time_elastic_2.0,rank_elastic_2.0,results_elastic_2.1,pages_elastic_2.1,siren_elastic_2.1,resp_time_elastic_2.1
0,stellantis,879786085,1383.0,70.0,"['879786085', '803902485', '500019898', '37911...",0.042709,0,2007.0,101.0,"['879786085', '897940953', '912425493', '91005...",0.044964,0,,,,
1,STELLANTIS,879786085,1383.0,70.0,"['879786085', '803902485', '500019898', '37911...",0.041482,0,2007.0,101.0,"['879786085', '897940953', '912425493', '91005...",0.046229,0,,,,
2,Stellantis,879786085,1383.0,70.0,"['879786085', '803902485', '500019898', '37911...",0.040855,0,2007.0,101.0,"['879786085', '897940953', '912425493', '91005...",0.047226,0,,,,


In [ ]:
for index, row in df_test.iterrows():
    (
        df_test[f"results_elastic_{SEARCH_VERSION}"][index],
        df_test[f"pages_elastic_{SEARCH_VERSION}"][index],
        df_test[f"siren_elastic_{SEARCH_VERSION}"][index],
        df_test[f"resp_time_elastic_{SEARCH_VERSION}"][index],
    ) = get_response(url_elastic, row["terms"])
    if index % 5 == 0: 
        time.sleep(0.5)

In [ ]:
df_test

In [ ]:
elastic_columns = [
    col
    for col in df_test.columns
    if "elastic" in col and ("result" in col or "pages" in col or "resp_time" in col)
]

In [ ]:
elastic_columns

In [ ]:
for col in elastic_columns:
    df_test[col] = df_test[col].astype("float64")

In [ ]:
df_test.dtypes

In [ ]:
df_test.describe()

In [ ]:
df_test.describe().to_csv(
    f"./output/describe/describe_{SEARCH_VERSION}.csv", header=True, index=True
)

## Ranks

In [ ]:
df_test[f"rank_elastic_{SEARCH_VERSION}"] = ""

In [ ]:
for ind, row in df_test.iterrows():
    if str(row["siren"]) in row[f"siren_elastic_{SEARCH_VERSION}"]:
        df_test[f"rank_elastic_{SEARCH_VERSION}"][ind] = row[
            f"siren_elastic_{SEARCH_VERSION}"
        ].index(str(row["siren"]))
    else:
        df_test[f"rank_elastic_{SEARCH_VERSION}"][ind] = -1

In [ ]:
df_test[f"rank_elastic_{SEARCH_VERSION}"] = df_test[
    f"rank_elastic_{SEARCH_VERSION}"
].astype("int32")

## KPIs

In [ ]:
fig = px.histogram(
    df_test.sort_values(by=[f"rank_elastic_{SEARCH_VERSION}"]),
    x=f"rank_elastic_{SEARCH_VERSION}",
    color_discrete_sequence=["indianred"],
    title="Distribution Elasticsearch des rangs du bon résultat",
)
fig.update_layout(bargap=0.5)
fig.update_xaxes(type="category")
fig.show()
plotly.offline.plot(fig, filename=f"./output/plots/rank_{SEARCH_VERSION}.html")

In [ ]:
rank_columns = [col for col in df_test.columns if "rank_elastic" in col]

In [ ]:
rank_columns

In [ ]:
fig = go.Figure()
rank_dict = {}
for rank in rank_columns:
    df_test[rank] = df_test[rank].astype("int32")
    rank_dict[rank] = df_test.sort_values(by=[rank])[rank]
    fig.add_trace(
        go.Histogram(
            histfunc="count",
            x=rank_dict[rank],
            name=rank,
        )
    )
fig.update_layout(
    title_text="Fréquence des rangs des résultats de la recherche",  # title of plot
    xaxis_title_text="Rang du résulat dans la page",  # xaxis label
    yaxis_title_text="Nombre de requêtes",  # yaxis label
    bargap=0.2,  # gap between bars of adjacent location coordinates
    bargroupgap=0.1,  # gap between bars of the same location coordinates
)
fig.update_xaxes(type="category")
fig.show()
# fig.write_image(f"./output/plots/rank_{SEARCH_VERSION}.png")
plotly.offline.plot(fig, filename=f"./output/plots/rank_war_{SEARCH_VERSION}.html")

In [ ]:
fig = px.pie(
    df_test,
    names=rank_columns[-1],
    hole=0.7,
    color=rank_columns[-1],
    title="Pourcentages des rangs du bon résultat dans la recherche elasticsearch",
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()
plotly.offline.plot(fig, filename=f"./output/plots/pie_{SEARCH_VERSION}.html")

### Nombre maximale de requête


In [ ]:
df_max = (
    df_test[f"results_elastic_{SEARCH_VERSION}"].value_counts(normalize=True) * 100
).reset_index()
df_max[df_max["index"] == "10000.0"].to_csv(
    f"./output/describe/max_requetes_{SEARCH_VERSION}.csv", header=True, index=True
)
df_max[df_max["index"] == "10000.0"]

### Sauvegarder les dataframes

In [ ]:
df_test.to_csv(f"./data/elastic_wars_{SEARCH_VERSION}.csv", header=True, index=False)

In [ ]:
elastic_columns = [col for col in df_test.columns if "elastic" in col]
columns_to_save = ["terms", "siren"]
columns_to_save = columns_to_save + elastic_columns[-10:]
columns_to_save

In [ ]:
df_test.to_csv(
    f"./data/elastic_wars.csv", header=True, index=False, columns=columns_to_save
)